In [1]:
! pip install optuna


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn import metrics as m
from sklearn.model_selection import train_test_split

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination

import shap
import optuna

In [3]:
data = pd.read_csv("../../datasets/credit.csv")
print(data.columns)
#data = data.reset_index().drop(columns=["index"])
n_features = data.shape[1]-1
n = data.shape[0]
data["default"] = (data["NumberOfTimes90DaysLate"] > 0).astype(int)
data.drop(columns=["NumberOfTimes90DaysLate", 'NumberOfTime60-89DaysPastDueNotWorse', 'SeriousDlqin2yrs',
                   'NumberOfTime30-59DaysPastDueNotWorse'], inplace=True)
n_features = data.shape[1]-1
n = data.shape[0]
data.rename(columns={feat: "".join(feat.split("-")) for feat in data.columns}, inplace=True)
target = "default"
features = data.columns.difference([target]).tolist()

data.head(10)

Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents', 'SeriousDlqin2yrs'],
      dtype='object')


,RevolvingUtilizationOfUnsecuredLines,age,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfDependents,default
0,0.006999,38,0.302150,5440.0,4,1,3,0
1,0.704592,63,0.471441,8000.0,9,1,0,0
2,0.063113,57,0.068586,5000.0,17,0,0,0
3,0.368397,68,0.296273,6250.0,16,2,0,0
4,1.000000,34,0.000000,3500.0,0,0,1,0
5,0.051799,63,0.770687,4700.0,16,1,1,0
6,0.648733,40,0.402060,6600.0,10,2,3,0
7,0.767270,33,0.338390,5800.0,10,0,3,0
8,0.190111,27,0.164605,3960.0,6,0,1,0
9,0.139989,65,0.258397,6400.0,6,1,0,0


In [4]:
data_tv, data_test = train_test_split(data, test_size=0.15, random_state=42,
                                      stratify=data[target])

data_train, data_val = train_test_split(data_tv, test_size=0.15, random_state=42,
                                      stratify=data_tv[target])

data_train[target].mean(), data_val[target].mean() , data_test[target].mean()

(0.18476190476190477, 0.18488972313467855, 0.18460925039872408)

In [ ]:
# подберём параметры n_estimators, learning rate, max_depth
GradientBoostingClassifier()

In [6]:
features

['DebtRatio',
 'MonthlyIncome',
 'NumberOfDependents',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberRealEstateLoansOrLines',
 'RevolvingUtilizationOfUnsecuredLines',
 'age']

In [17]:
params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}

def score(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    return m.roc_auc_score(y, y_pred)

def objective(trial, features, target):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    clf = GradientBoostingClassifier(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    clf.fit(data_train[features], data_train[target])
    return score(clf, data_val[features], data_val[target])

In [18]:
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: objective(trial, features, target), n_trials=50, show_progress_bar=True)
df = study.trials_dataframe()

[I 2025-04-15 23:11:42,591] A new study created in memory with name: no-name-881f4f3f-baed-4725-982a-763194f3feb6
[I 2025-04-15 23:11:53,121] Trial 0 finished with value: 0.7226642294170765 and parameters: {'n_estimators': 64, 'max_depth': 9, 'learning_rate': 0.0014889385648724077}. Best is trial 0 with value: 0.7226642294170765.
[I 2025-04-15 23:12:07,054] Trial 1 finished with value: 0.7439251407847711 and parameters: {'n_estimators': 96, 'max_depth': 9, 'learning_rate': 0.4106997985910941}. Best is trial 1 with value: 0.7439251407847711.
[I 2025-04-15 23:12:10,476] Trial 2 finished with value: 0.7063099632074672 and parameters: {'n_estimators': 21, 'max_depth': 9, 'learning_rate': 1.3930156276278107e-05}. Best is trial 1 with value: 0.7439251407847711.
[I 2025-04-15 23:12:19,910] Trial 3 finished with value: 0.7654424893845214 and parameters: {'n_estimators': 65, 'max_depth': 9, 'learning_rate': 0.10362913159912819}. Best is trial 3 with value: 0.7654424893845214.
[I 2025-04-15 23:1

In [19]:
study.best_params

{'n_estimators': 94, 'max_depth': 3, 'learning_rate': 0.13962813919623543}

In [ ]:
{'n_estimators': 66, 'max_depth': 3, 'learning_rate': 0.0971}

In [14]:
clf = GradientBoostingClassifier(random_state=42, **study.best_params)

In [15]:
clf.__class__

sklearn.ensemble._gb.GradientBoostingClassifier

In [16]:
GradientBoostingClassifier

sklearn.ensemble._gb.GradientBoostingClassifier